In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [9]:
config = json.loads(open('nyu-2451-34514-geojson.json').read())
config.

{'type': 'FeatureCollection',
 'totalFeatures': 346,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34514.1',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-73.89403299999996, 40.68513899999982],
       [-73.89271399999998, 40.68691999999983],
       [-73.89155399999999, 40.68824899999983],
       [-73.890483, 40.68846899999981],
       [-73.88979199999999, 40.68802099999983],
       [-73.89002199999999, 40.68861999999983],
       [-73.89037699999999, 40.68981599999982],
       [-73.89005299999997, 40.69022599999982],
       [-73.88918399999999, 40.69074899999982],
       [-73.88611, 40.69128299999983],
       [-73.88475699999996, 40.69125099999983],
       [-73.88276799999998, 40.689627999999814],
       [-73.88316399999997, 40.68902399999982],
       [-73.88327499999998, 40.688253999999795],
       [-73.88375699999999, 40.6878789999998],
       [-73.88377699999998, 40.68786299999983],
       [-73.88395499999999, 40.68776899999981],
       [-73.88566499999999, 4

In [10]:
landmarks_data = config['features']

In [11]:
landmarks_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34514.1',
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-73.89403299999996, 40.68513899999982],
     [-73.89271399999998, 40.68691999999983],
     [-73.89155399999999, 40.68824899999983],
     [-73.890483, 40.68846899999981],
     [-73.88979199999999, 40.68802099999983],
     [-73.89002199999999, 40.68861999999983],
     [-73.89037699999999, 40.68981599999982],
     [-73.89005299999997, 40.69022599999982],
     [-73.88918399999999, 40.69074899999982],
     [-73.88611, 40.69128299999983],
     [-73.88475699999996, 40.69125099999983],
     [-73.88276799999998, 40.689627999999814],
     [-73.88316399999997, 40.68902399999982],
     [-73.88327499999998, 40.688253999999795],
     [-73.88375699999999, 40.6878789999998],
     [-73.88377699999998, 40.68786299999983],
     [-73.88395499999999, 40.68776899999981],
     [-73.88566499999999, 40.68686899999983],
     [-73.88762499999997, 40.686001999999824],
     [-73.88829399999999, 40.685174999999

In [82]:
# define the dataframe columns
column_names = ['statefp', 'areaid','fullname','Latitude', 'Longitude'] 

# instantiate the dataframe
landmarks = pd.DataFrame(columns=column_names)
landmarks

,statefp,areaid,fullname,Latitude,Longitude


In [100]:
for data in landmarks_data:
    st = data['properties']['statefp'] 
    ar = data['properties']['areaid']
    fn = data['properties']['fullname'] 
    lt = data['properties']['intptlat']
    ln = data['properties']['intptlon']
        
    landmarks = landmarks.append({'statefp': st,
                                          'areaid': ar,
                                           'fullname':fn, 
                                          'Latitude': lt,
                                          'Longitude': ln}, ignore_index=True)

In [130]:
landmarks.head(20)

,statefp,areaid,fullname,Latitude,Longitude
0,36,1102411323286,Highland Park,+40.6853885,-73.8912170
1,36,110391537892,Rat Is,+40.8554841,-73.7811514
2,36,110391537913,Ewen Park,+40.8821342,-73.9100443
3,36,110391537916,Hart Is,+40.8541547,-73.7697254
4,36,110391537918,High Is,+40.8595313,-73.7860450
5,36,110391537929,Bronx Community Colg,+40.8574375,-73.9131413
6,36,110391537936,Bronx Park,+40.8578720,-73.8755467
7,36,110391537938,Goose Is,+40.8657054,-73.8186936
8,36,110391537939,Goose Is,+40.8664547,-73.8176223
9,36,110391537941,Hylan Park,+40.8390156,-73.8876981


In [65]:
CLIENT_ID = 'AQ3QJ423ICYYSQ0XQCVBXCKUI4NSRDUZAK1QIE4T2KPQUAQD' # your Foursquare ID
CLIENT_SECRET = 'SX2CZZC13GPTCQXAE4VVQHMZ42N4RENPM1KYKURLTNPXE4N3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AQ3QJ423ICYYSQ0XQCVBXCKUI4NSRDUZAK1QIE4T2KPQUAQD
CLIENT_SECRET:SX2CZZC13GPTCQXAE4VVQHMZ42N4RENPM1KYKURLTNPXE4N3


The geograpical coordinate of Highland Park are 42.1816919, -87.8003438.


In [144]:
landmarks.loc[0, 'fullname']

'Highland Park'

In [146]:
Location_latitude = landmarks.loc[0, 'Latitude'] #  latitude value
Location_longitude = landmarks.loc[0, 'Longitude'] # longitude value

Location_name = landmarks.loc[0, 'fullname'] # location name

print('Latitude and longitude values of {} are {}, {}.'.format(Location_name, 
                                                               Location_latitude, 
                                                               Location_longitude))

Latitude and longitude values of Highland Park are +40.6853885, -73.8912170.


In [155]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Location_latitude, 
    Location_longitude, 
    radius, 
    LIMIT)
url 



'https://api.foursquare.com/v2/venues/explore?&client_id=AQ3QJ423ICYYSQ0XQCVBXCKUI4NSRDUZAK1QIE4T2KPQUAQD&client_secret=SX2CZZC13GPTCQXAE4VVQHMZ42N4RENPM1KYKURLTNPXE4N3&v=20180605&ll=+40.6853885,-73.8912170&radius=1000&limit=100'

In [156]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d21d983c53093002c6a9f44'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Cypress Hills',
  'headerFullLocation': 'Cypress Hills, Brooklyn',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 40.694388509000014,
    'lng': -73.87937050423287},
   'sw': {'lat': 40.67638849099999, 'lng': -73.90306349576713}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4cddebf7aba88cfa802b3ed7',
       'name': 'Highland Park',
       'location': {'lat': 40.682002318507095,
        'lng': -73.88917283223061,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.682002318507095,
          'lng': -

In [157]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [158]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

,name,categories,lat,lng
0,Highland Park,Park,40.682002,-73.889173
1,Caterina Pizza,Pizza Place,40.680558,-73.882778
2,"Macorix Restaurant, Bar & Grill",Latin American Restaurant,40.678523,-73.896468
3,Little Caesars Pizza,Pizza Place,40.678613,-73.888647
4,Rico Chimi Cafeteria,Sandwich Place,40.677456,-73.886389
5,Caoba,Latin American Restaurant,40.678566,-73.897026
6,North Brooklyn YMCA,Gym / Fitness Center,40.682066,-73.889165
7,Baskin-Robbins,Ice Cream Shop,40.678243,-73.897858
8,Bravo Supermarkets,Grocery Store,40.681354,-73.883164
9,CTown Supermarkets,Supermarket,40.678821,-73.888548


In [159]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.
